# Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012
from missingData.toolkits import toolkits

2025-03-11 20:27:30.116930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741735650.169067    1057 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741735650.184632    1057 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 20:27:30.317074: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/pypots/nn/functional/cuda.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Loading database

### StandardScaler

In [2]:
set_random_seed()
physionet2012_dataset_standard = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset_standard.keys())

2025-03-10 22:32:47 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-10 22:32:47 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-10 22:32:47 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-10 22:32:47 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-10 22:32:48 [INFO]: Loaded successfully!
2025-03-10 22:33:02 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-10 22:33:02 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-10 22:33:02 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
scaler_standard = physionet2012_dataset_standard["scaler"]

In [4]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = toolkits.separating_dataset(physionet2012_dataset_standard)

### MinMaxScaler

In [5]:
set_random_seed()
physionet2012_dataset_minmax = preprocess_physionet2012(subset="all", rate=0.1, normalization=2)
print(physionet2012_dataset_minmax.keys())

2025-03-10 22:33:10 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-03-10 22:33:10 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-03-10 22:33:10 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-03-10 22:33:10 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-03-10 22:33:10 [INFO]: Loaded successfully!
2025-03-10 22:33:24 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-10 22:33:24 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-03-10 22:33:24 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [6]:
scaler_minmax = physionet2012_dataset_minmax["scaler"]

In [7]:
dataset_for_training_minmax, dataset_for_validating_minmax, dataset_for_testing_minmax, dataset_for_testing_ori_minmax = toolkits.separating_dataset(physionet2012_dataset_minmax)

# Creating indicating mask

### StandardScaler

In [8]:
indicating_mask_variable_standard, test_X_ori_variable_standard = toolkits.components_mae(dataset_for_testing_ori_standard, dataset_for_testing_standard)
indicating_mask_variable_standard = toolkits.pre_reshape(indicating_mask_variable_standard)
test_X_ori_variable_standard = toolkits.pre_reshape(test_X_ori_variable_standard)
test_X_ori_variable_standard_ori = toolkits.desnormalization(test_X_ori_variable_standard, scaler_standard)
indicating_mask_variable_standard = toolkits.reshape_variable(indicating_mask_variable_standard)
test_X_ori_variable_standard = toolkits.reshape_variable(test_X_ori_variable_standard)
test_X_ori_variable_standard_ori = toolkits.reshape_variable(test_X_ori_variable_standard_ori)

### MinMaxScaler 

In [9]:
indicating_mask_variable_minmax, test_X_ori_variable_minmax = toolkits.components_mae(dataset_for_testing_ori_minmax, dataset_for_testing_minmax)
indicating_mask_variable_minmax = toolkits.pre_reshape(indicating_mask_variable_minmax)
test_X_ori_variable_minmax = toolkits.pre_reshape(test_X_ori_variable_minmax)
test_X_ori_variable_minmax_ori = toolkits.desnormalization(test_X_ori_variable_minmax, scaler_minmax)
indicating_mask_variable_minmax = toolkits.reshape_variable(indicating_mask_variable_minmax)
test_X_ori_variable_minmax = toolkits.reshape_variable(test_X_ori_variable_minmax)
test_X_ori_variable_minmax_ori = toolkits.reshape_variable(test_X_ori_variable_minmax_ori)

# Inicialize the models

### SAITS

<h4> Inicialize new model (Standard)</h4>

In [10]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-10 22:34:19 [INFO]: No given device, using default device: cuda
2025-03-10 22:34:19 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250310_T223419
2025-03-10 22:34:19 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250310_T223419/tensorboard
2025-03-10 22:34:19 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
saits_standard = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-04 16:52:24 [INFO]: No given device, using default device: cuda
2025-03-04 16:52:24 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-03-04 16:52:24 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize new model (MinMax)</h4>

In [11]:
saits_minmax = SAITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-03-10 22:34:24 [INFO]: No given device, using default device: cuda
2025-03-10 22:34:24 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250310_T223424
2025-03-10 22:34:24 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250310_T223424/tensorboard
2025-03-10 22:34:24 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
saits_minmax = SAITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

### BRITS

<h4> Inicialize new model (Standard)</h4>

In [ ]:
brits_standard = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-24 21:15:49 [INFO]: No given device, using default device: cuda
2025-02-24 21:15:49 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250224_T211549
2025-02-24 21:15:49 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250224_T211549/tensorboard
2025-02-24 21:15:49 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
brits_standard = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-25 21:04:33 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:33 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:33 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize new model (MinMax)</h4>

In [ ]:
brits_minmax = BRITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
brits_minmax = BRITS(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

### USGAN

<h4> Inicialize new model (Standard)</h4>

In [ ]:
us_gan_standard = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-24 21:26:19 [INFO]: No given device, using default device: cuda
2025-02-24 21:26:19 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250224_T212619
2025-02-24 21:26:19 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250224_T212619/tensorboard
2025-02-24 21:26:19 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
us_gan_standard = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-25 21:04:37 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:37 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:37 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize new model (MinMax)</h4>

In [ ]:
us_gan_minmax = USGAN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
us_gan_minmax = USGAN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

### GPVAE

<h4> Inicialize new model (Standard)</h4>

In [ ]:
gp_vae_standard = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-24 21:38:42 [INFO]: No given device, using default device: cuda
2025-02-24 21:38:42 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250224_T213842
2025-02-24 21:38:42 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250224_T213842/tensorboard
2025-02-24 21:38:42 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
gp_vae_standard = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-25 21:04:42 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:42 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:42 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize new model (MinMax)</h4>

In [ ]:
gp_vae_minmax = GPVAE(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
gp_vae_minmax= GPVAE(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

### MRNN

<h4> Inicialize new model (Standard)</h4>

In [ ]:
mrnn_standard = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-24 21:40:01 [INFO]: No given device, using default device: cuda
2025-02-24 21:40:01 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250224_T214001
2025-02-24 21:40:01 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250224_T214001/tensorboard
2025-02-24 21:40:01 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize existing model (Standard)</h4>

In [ ]:
mrnn_standard = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-25 21:04:45 [INFO]: No given device, using default device: cpu
2025-02-25 21:04:45 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-25 21:04:45 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize new model (MinMax)</h4>

In [ ]:
mrnn_minmax = MRNN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

<h4> Inicialize existing model (MinMax)</h4>

In [ ]:
mrnn_minmax = MRNN(
    n_steps=physionet2012_dataset_minmax['n_steps'],
    n_features=physionet2012_dataset_minmax['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

# Train/Load the models

### SAITS

<h4>Train - Standard Scaler </h4>

In [12]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-03-10 22:34:50 [INFO]: Epoch 001 - training loss (MSE): 0.7691, validation MSE: 6.7329
2025-03-10 22:34:53 [INFO]: Epoch 002 - training loss (MSE): 0.5444, validation MSE: 6.6818
2025-03-10 22:34:56 [INFO]: Epoch 003 - training loss (MSE): 0.4779, validation MSE: 6.6747
2025-03-10 22:34:59 [INFO]: Epoch 004 - training loss (MSE): 0.4593, validation MSE: 6.6579
2025-03-10 22:35:04 [INFO]: Epoch 005 - training loss (MSE): 0.4215, validation MSE: 6.6589
2025-03-10 22:35:17 [INFO]: Epoch 006 - training loss (MSE): 0.4239, validation MSE: 6.6434
2025-03-10 22:35:30 [INFO]: Epoch 007 - training loss (MSE): 0.4114, validation MSE: 6.7782
2025-03-10 22:35:42 [INFO]: Epoch 008 - training loss (MSE): 0.4424, validation MSE: 6.6446
2025-03-10 22:35:55 [INFO]: Epoch 009 - training loss (MSE): 0.4374, validation MSE: 6.6392
2025-03-10 22:36:07 [INFO]: Epoch 010 - training loss (MSE): 0.3752, validation MSE: 6.6284
2025-03-10 22:36:07 [INFO]: Finished training. The best model is from epoch#10.


<h4>Train - MinMax Scaler</h4>

In [13]:
saits_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

2025-03-10 22:36:24 [INFO]: Epoch 001 - training loss (MSE): 0.0368, validation MSE: 0.0079
2025-03-10 22:36:36 [INFO]: Epoch 002 - training loss (MSE): 0.0163, validation MSE: 0.0075
2025-03-10 22:36:49 [INFO]: Epoch 003 - training loss (MSE): 0.0123, validation MSE: 0.0055
2025-03-10 22:37:02 [INFO]: Epoch 004 - training loss (MSE): 0.0100, validation MSE: 0.0043
2025-03-10 22:37:14 [INFO]: Epoch 005 - training loss (MSE): 0.0080, validation MSE: 0.0039
2025-03-10 22:37:27 [INFO]: Epoch 006 - training loss (MSE): 0.0069, validation MSE: 0.0036
2025-03-10 22:37:39 [INFO]: Epoch 007 - training loss (MSE): 0.0063, validation MSE: 0.0034
2025-03-10 22:37:52 [INFO]: Epoch 008 - training loss (MSE): 0.0059, validation MSE: 0.0035
2025-03-10 22:38:04 [INFO]: Epoch 009 - training loss (MSE): 0.0056, validation MSE: 0.0032
2025-03-10 22:38:17 [INFO]: Epoch 010 - training loss (MSE): 0.0054, validation MSE: 0.0032
2025-03-10 22:38:17 [INFO]: Finished training. The best model is from epoch#9.
2

<h4> Load - Standard Scaler </h4>

In [ ]:
saits_standard.load("tutorial_results/imputation/saits/standard_scaler/SAITS.pypots")

<h4> Load - Min/Max Scaler </h4>

In [ ]:
saits_minmax.load("tutorial_results/imputation/saits/20250226_T191938/SAITS.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:324: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-2

### BRITS

<h4> Train - Standard Scaler </h4>

In [ ]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-02-24 21:17:11 [INFO]: Epoch 001 - training loss: 0.9419, validation loss: 6.7858
2025-02-24 21:17:11 [INFO]: Saved the model to tutorial_results/imputation/brits/20250224_T211549/BRITS_epoch1_loss6.78576402142644.pypots
2025-02-24 21:17:57 [INFO]: Epoch 002 - training loss: 0.7344, validation loss: 6.7420
2025-02-24 21:17:57 [INFO]: Saved the model to tutorial_results/imputation/brits/20250224_T211549/BRITS_epoch2_loss6.742043674240509.pypots
2025-02-24 21:18:43 [INFO]: Epoch 003 - training loss: 0.6836, validation loss: 6.7305
2025-02-24 21:18:43 [INFO]: Saved the model to tutorial_results/imputation/brits/20250224_T211549/BRITS_epoch3_loss6.730484957744678.pypots
2025-02-24 21:19:29 [INFO]: Epoch 004 - training loss: 0.6590, validation loss: 6.7263
2025-02-24 21:19:29 [INFO]: Saved the model to tutorial_results/imputation/brits/20250224_T211549/BRITS_epoch4_loss6.726297639558712.pypots
2025-02-24 21:20:17 [INFO]: Epoch 005 - training loss: 0.6437, validation loss: 6.7247
2025-0

<h4> Train - MinMax Scaler</h4>

In [ ]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

<h4>Load - Standard Scaler</h4>

In [ ]:
brits_standard.load("tutorial_results/imputation/brits/standard_scaler/BRITS.pypots")

2025-02-25 21:05:09 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/standard_scaler/BRITS.pypots


<h4>Load - Min/Max Scaler</h4>

In [ ]:
brits_minmax.load("tutorial_results/imputation/brits/20250210_T210425/BRITS.pypots")

NameError: name 'brits' is not defined

### USGAN

<h4>Train - Standard Scaler </h4>

In [ ]:
us_gan_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-02-24 21:27:53 [INFO]: Epoch 001 - generator training loss: 0.4347, discriminator training loss: 0.1896, validation loss: 6.7770
2025-02-24 21:27:53 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250224_T212619/USGAN_epoch1_loss6.776960478474696.pypots
2025-02-24 21:29:00 [INFO]: Epoch 002 - generator training loss: 0.3748, discriminator training loss: 0.0544, validation loss: 6.7267
2025-02-24 21:29:00 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250224_T212619/USGAN_epoch2_loss6.726737344016631.pypots
2025-02-24 21:30:09 [INFO]: Epoch 003 - generator training loss: 0.3420, discriminator training loss: 0.0366, validation loss: 6.7000
2025-02-24 21:30:09 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250224_T212619/USGAN_epoch3_loss6.699957043429216.pypots
2025-02-24 21:31:17 [INFO]: Epoch 004 - generator training loss: 0.3202, discriminator training loss: 0.0309, validation loss: 6.6893
2025-02-24 21:31:17 [INFO]: Saved the mod

<h4> Train - MinMax Scaler</h4>

In [ ]:
us_gan_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

<h4>Load - Standard Scaler</h4>

In [ ]:
us_gan_standard.load("tutorial_results/imputation/us_gan/standard_scaler/USGAN.pypots")

2025-02-25 21:05:34 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/standard_scaler/USGAN.pypots


<h4>Load - Min/Max Scaler</h4>

In [ ]:
us_gan_minmax.load("tutorial_results/imputation/us_gan/20250205_T212325/USGAN.pypots")

### GPVAE

<h4>Train - Standard Scaler</h4>

In [ ]:
gp_vae_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-02-24 21:39:04 [INFO]: Epoch 001 - training loss: 25923.7769, validation loss: 7.0152
2025-02-24 21:39:04 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250224_T213842/GPVAE_epoch1_loss7.015189884106318.pypots
2025-02-24 21:39:07 [INFO]: Epoch 002 - training loss: 22873.1363, validation loss: 6.9989
2025-02-24 21:39:07 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250224_T213842/GPVAE_epoch2_loss6.998940398792426.pypots
2025-02-24 21:39:10 [INFO]: Epoch 003 - training loss: 22842.5329, validation loss: 6.9768
2025-02-24 21:39:10 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250224_T213842/GPVAE_epoch3_loss6.976767674088478.pypots
2025-02-24 21:39:13 [INFO]: Epoch 004 - training loss: 22827.7529, validation loss: 6.9652
2025-02-24 21:39:13 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250224_T213842/GPVAE_epoch4_loss6.965172406037649.pypots
2025-02-24 21:39:16 [INFO]: Epoch 005 - training loss: 22826.0822, valid

<h4>Train - MinMax Scaler</h4>

In [ ]:
gp_vae_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

<h4>Load - Standard Scaler</h4>

In [ ]:
gp_vae_standard.load("tutorial_results/imputation/gp_vae/standard_scaler/GPVAE.pypots")

2025-02-25 21:05:46 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/standard_scaler/GPVAE.pypots


<h4>Load - MinMax Scaler</h4>

In [ ]:
gp_vae_minmax.load("tutorial_results/imputation/gp_vae/20250205_T214223/GPVAE.pypots")

### MRNN

<h4>Train - Standard Scaler</h4>

In [ ]:
mrnn_standard.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-02-24 21:40:42 [INFO]: Epoch 001 - training loss: 0.7549, validation loss: 7.3190
2025-02-24 21:40:42 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250224_T214001/MRNN_epoch1_loss7.318970077236494.pypots
2025-02-24 21:40:56 [INFO]: Epoch 002 - training loss: 0.5305, validation loss: 7.2713
2025-02-24 21:40:56 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250224_T214001/MRNN_epoch2_loss7.271315918366114.pypots
2025-02-24 21:41:11 [INFO]: Epoch 003 - training loss: 0.4909, validation loss: 7.2550
2025-02-24 21:41:11 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250224_T214001/MRNN_epoch3_loss7.254956531524658.pypots
2025-02-24 21:41:26 [INFO]: Epoch 004 - training loss: 0.4781, validation loss: 7.2454
2025-02-24 21:41:26 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250224_T214001/MRNN_epoch4_loss7.245413625240326.pypots
2025-02-24 21:41:41 [INFO]: Epoch 005 - training loss: 0.4575, validation loss: 7.2425
2025-02-24 21

<h4>Train - MinMax Scaler</h4>

In [ ]:
mrnn_minmax.fit(train_set=dataset_for_training_minmax, val_set=dataset_for_validating_minmax)

<h4>Load - Standard Scaler</h4>

In [ ]:
mrnn_standard.load("tutorial_results/imputation/mrnn/standard_scaler/MRNN.pypots")

2025-02-25 21:05:52 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/standard_scaler/MRNN.pypots


<h4>Load - MinMax Scaler</h4>

In [ ]:
mrnn_minmax.load("tutorial_results/imputation/mrnn/20250205_T221050/MRNN.pypots")

# Testing stage

### SAITS

#### Standard Scaler

In [ ]:
saits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, saits_standard)
saits_imputation_standard = toolkits.pre_reshape(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.desnormalization(saits_imputation_standard, scaler_standard)
saits_imputation_standard = toolkits.reshape_variable(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.reshape_variable(saits_imputation_standard_ori)

#### MinMax Scaler

In [ ]:
saits_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, saits_minmax)
saits_imputation_minmax = toolkits.pre_reshape(saits_imputation_minmax)
saits_imputation_minmax_ori = toolkits.desnormalization(saits_imputation_minmax, scaler_minmax)
saits_imputation_minmax = toolkits.reshape_variable(saits_imputation_minmax)
saits_imputation_minmax_ori = toolkits.reshape_variable(saits_imputation_minmax_ori)

### BRITS

#### Standard Scaler

In [ ]:
brits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, brits_standard)
brits_imputation_standard = toolkits.pre_reshape(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.desnormalization(brits_imputation_standard, scaler_standard)
brits_imputation_standard = toolkits.reshape_variable(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.reshape_variable(brits_imputation_standard_ori)

#### MinMax Scaler

In [ ]:
brits_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, brits_minmax)
brits_imputation_minmax = toolkits.pre_reshape(brits_imputation_minmax)
brits_imputation_minmax_ori = toolkits.desnormalization(brits_imputation_minmax, scaler_minmax) 
brits_imputation_minmax = toolkits.reshape_variable(brits_imputation_minmax)
brits_imputation_minmax_ori = toolkits.reshape_variable(brits_imputation_minmax_ori)  

### USGAN

#### Standard Scaler

In [ ]:
usgan_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, us_gan_standard)
usgan_imputation_standard = toolkits.pre_reshape(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.desnormalization(usgan_imputation_standard, scaler_standard)
usgan_imputation_standard = toolkits.reshape_variable(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.reshape_variable(usgan_imputation_standard_ori)

#### MinMax Scaler

In [ ]:
usgan_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, us_gan_minmax)
usgan_imputation_minmax = toolkits.pre_reshape(usgan_imputation_minmax)
usgan_imputation_minmax_ori = toolkits.desnormalization(brits_imputation_minmax, scaler_minmax)
usgan_imputation_minmax = toolkits.reshape_variable(usgan_imputation_minmax)
usgan_imputation_minmax_ori = toolkits.reshape_variable(usgan_imputation_minmax_ori)

### GPVAE

#### Standard Scaler

In [ ]:
gpvae_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, gp_vae_standard)
gpvae_imputation_standard = toolkits.pre_reshape(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.desnormalization(gpvae_imputation_standard, scaler_standard)
gpvae_imputation_standard = toolkits.reshape_variable(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.reshape_variable(gpvae_imputation_standard_ori) 

#### MinMax Scaler

In [ ]:
gpvae_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, gp_vae_minmax)
gpvae_imputation_minmax = toolkits.pre_reshape(gpvae_imputation_minmax)
gpvae_imputation_minmax_ori = toolkits.desnormalization(gpvae_imputation_minmax, scaler_minmax)
gpvae_imputation_minmax = toolkits.reshape_variable(gpvae_imputation_minmax)
gpvae_imputation_minmax_ori = toolkits.reshape_variable(gpvae_imputation_minmax_ori) 

### MRNN

#### Standard Scaler

In [ ]:
mrnn_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, mrnn_standard)
mrnn_imputation_standard = toolkits.pre_reshape(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.desnormalization(mrnn_imputation_standard, scaler_standard)
mrnn_imputation_standard = toolkits.reshape_variable(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.reshape_variable(mrnn_imputation_standard_ori) 

#### MinMax Scaler

In [ ]:
mrnn_imputation_minmax = toolkits.model_imputation(dataset_for_testing_minmax, mrnn_minmax)
mrnn_imputation_minmax = toolkits.pre_reshape(mrnn_imputation_minmax)
mrnn_imputation_minmax_ori = toolkits.desnormalization(mrnn_imputation_minmax, scaler_minmax)
mrnn_imputation_minmax = toolkits.reshape_variable(mrnn_imputation_minmax)
mrnn_imputation_minmax_ori = toolkits.reshape_variable(mrnn_imputation_minmax_ori) 

# Calculate mean absolute error

### SAITS

#### Standard scaler (C/Normalização)

In [16]:
testing_mae_saits_variables_standard = toolkits.calculate_mae(saits_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

#### Standard scaler (S/Normalização)

In [ ]:
testing_mae_saits_variables_standard_ori = toolkits.calculate_mae(saits_imputation_standard_ori, test_X_ori_variable_standard_ori, indicating_mask_variable_standard)

#### MinMax scaler (C/Normalização)

In [17]:
testing_mae_saits_variables_minmax = toolkits.calculate_mae(saits_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

#### MinMax scaler (S/Normalização)

In [ ]:
testing_mae_saits_variables_minmax_ori = toolkits.calculate_mae(saits_imputation_minmax_ori, test_X_ori_variable_minmax_ori, indicating_mask_variable_minmax)

### BRITS

#### Standard Scaler (C/Normalização)

In [ ]:
testing_mae_brits_variables_standard = toolkits.calculate_mae(brits_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

#### Standard Scaler (S/Normalização)

In [ ]:
testing_mae_brits_variables_standard_ori = toolkits.calculate_mae(brits_imputation_standard_ori, test_X_ori_variable_standard_ori, indicating_mask_variable_standard)

#### MinMax Scaler (C/Normalização)

In [ ]:
testing_mae_brits_variables_minmax = toolkits.calculate_mae(brits_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

#### MinMax Scaler (S/Normalização)

In [ ]:
testing_mae_brits_variables_minmax_ori = toolkits.calculate_mae(brits_imputation_minmax_ori, test_X_ori_variable_minmax_ori, indicating_mask_variable_minmax)

### USGAN

#### Standard Scaler (C/Normalização)

In [ ]:
testing_mae_usgan_variables_standard = toolkits.calculate_mae(usgan_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

#### Standard Scaler (S/Normalização)

In [ ]:
testing_mae_usgan_variables_standard_ori = toolkits.calculate_mae(usgan_imputation_standard_ori, test_X_ori_variable_standard_ori, indicating_mask_variable_standard)

#### MinMax Scaler (C/Normalização)

In [ ]:
testing_mae_usgan_variables_minmax = toolkits.calculate_mae(usgan_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

#### MinMax Scaler (S/Normalização)

In [ ]:
testing_mae_usgan_variables_minmax_ori = toolkits.calculate_mae(usgan_imputation_minmax_ori, test_X_ori_variable_minmax_ori, indicating_mask_variable_minmax)

### GPVAE

#### Standard Scaler (C/Normalização)

In [ ]:
testing_mae_gpvae_variables_standard = toolkits.calculate_mae(gpvae_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

#### Standard Scaler (S/Normalização)

In [ ]:
testing_mae_gpvae_variables_standard_ori = toolkits.calculate_mae(gpvae_imputation_standard_ori, test_X_ori_variable_standard_ori, indicating_mask_variable_standard)

#### MinMax Scaler (C/Normalização)

In [ ]:
testing_mae_gpvae_variables_minmax = toolkits.calculate_mae(gpvae_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

#### MinMax Scaler (S/Normalização)

In [ ]:
testing_mae_gpvae_variables_minmax_ori = toolkits.calculate_mae(gpvae_imputation_minmax_ori, test_X_ori_variable_minmax_ori, indicating_mask_variable_minmax)

### MRNN

#### Standard Scaler (C/Normalização)

In [ ]:
testing_mae_mrnn_variables_standard = toolkits.calculate_mae(mrnn_imputation_standard, test_X_ori_variable_standard, indicating_mask_variable_standard)

#### Standard Scaler (S/Normalização)

In [ ]:
testing_mae_mrnn_variables_standard_ori = toolkits.calculate_mae(mrnn_imputation_standard_ori, test_X_ori_variable_standard_ori, indicating_mask_variable_standard)

#### MinMax Scaler (C/Normalização)

In [ ]:
testing_mae_mrnn_variables_minmax = toolkits.calculate_mae(mrnn_imputation_minmax, test_X_ori_variable_minmax, indicating_mask_variable_minmax)

#### MinMax Scaler (S/Normalização)

In [ ]:
testing_mae_mrnn_variables_minmax_ori = toolkits.calculate_mae(mrnn_imputation_minmax_ori, test_X_ori_variable_minmax_ori, indicating_mask_variable_minmax)

# Results mean absolute error

In [18]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [19]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

#### SAITS

<h4>Standard Scaler (C/Normalização)</h4>

In [20]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 37.78297266364078
ALT : 270.4063470760967
AST : 400.73789268472706
Albumin : 0.37630310573165954
BUN : 5.041922862972568
Bilirubin : 1.462853714307099
Cholesterol : 40.12622451782026
Creatinine : 0.3069420256832965
DiasABP : 2.7704566013622594
FiO2 : 0.08620956248667794
GCS : 0.9567408451844454
Glucose : 35.11931412160936
HCO3 : 2.0015247896843125
HCT : 2.6748668442108534
HR : 4.738258661774259
K : 0.4027210196933226
Lactate : 1.1258680202247207
MAP : 3.003517885020544
MechVent : 0.012462921181450714
Mg : 0.2673795162468418
NIDiasABP : 2.679144880002555
NIMAP : 2.2011852084165207
NISysABP : 5.161661609334132
Na : 2.1882702717229923
PaCO2 : 4.549572724025382
PaO2 : 40.3313324594816
Platelets : 37.037300268262754
RespRate : 2.5052943566163246
SaO2 : 1.3193937473542672
SysABP : 5.728127288963052
Temp : 0.38344286305542885
TroponinI : 8.488083906173282
TroponinT : 1.0154387491644747
Urine : 58.69247400513298
WBC : 5.291650234308072
Weigh

In [21]:
df_saits_mae_standard = toolkits.create_table(testing_mae_saits_variables_standard, subgroups, variables)

In [22]:
df_saits_mae_standard

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,37.782973,45.908010,44.232028,0.000000,37.582313,90.367758,26.574981,22.825367,86.875150,56.892576,35.112586,60.076765,38.924773,27.006956,21.614137,35.656461,21.327258
1,ALT,270.406347,183.764102,176.910857,0.000000,195.893023,247.790016,290.689956,166.770505,187.666144,199.008963,233.592981,117.087886,152.725655,188.666646,197.012515,218.366870,376.077477
2,AST,400.737893,395.870124,292.496996,0.000000,231.160547,371.550484,660.623368,213.677841,392.764369,469.255462,346.102288,1023.089504,197.998634,522.010286,329.881948,43.184342,137.905937
3,Albumin,0.376303,0.414369,0.418830,0.000000,0.329526,0.422010,0.384738,0.486044,0.438151,0.446954,0.449860,0.652044,0.521462,0.511398,0.385117,0.343609,0.265561
4,BUN,5.041923,5.060138,4.804880,0.000000,5.125648,5.193089,4.919051,4.029728,6.121344,4.160634,5.540460,5.120281,4.822348,4.370578,4.105110,6.027658,5.077937
5,Bilirubin,1.462854,1.715884,1.350716,0.000000,1.662760,1.732945,1.693838,1.430183,1.771845,1.587349,1.835563,4.056222,1.963607,1.194848,1.646914,1.072857,1.600829
6,Cholesterol,40.126225,57.474820,36.346437,0.000000,34.275347,51.160950,40.131332,0.000000,41.187933,29.733724,29.371134,0.580627,36.292580,48.568815,25.462769,4.860565,0.000000
7,Creatinine,0.306942,0.245940,0.286378,0.000000,0.253128,0.292305,0.349482,0.261187,0.372952,0.247486,0.298109,0.201534,0.321650,0.223180,0.271594,0.234064,0.242901
8,DiasABP,2.770457,2.758157,2.724575,2.625834,2.801946,2.674256,3.191496,2.415311,2.747072,2.711476,2.787839,3.141755,2.595952,2.780184,2.668520,2.909210,2.375226
9,FiO2,0.086210,0.084405,0.090973,0.000000,0.087709,0.083642,0.094766,0.083189,0.085608,0.081931,0.082010,0.084822,0.084759,0.083246,0.085762,0.086346,0.101142


<h5>Minimum MAE value in each subgroup</h5>

In [23]:
toolkits.min_value_in_subgroup(df_saits_mae_standard, subgroups, variables)

General
MechVent
0.012462921181450714
--------------------
Female
MechVent
0.01182785045626394
--------------------
Male
MechVent
0.013049005193913223
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
MechVent
0.01248656810672432
--------------------
-65
MechVent
0.012519296893875705
--------------------
ICUType 1
MechVent
0.010791735683413642
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
MechVent
0.012368280552701387
--------------------
ICUType 4
MechVent
0.012251276172731964
--------------------
Undefined classification
MechVent
0.01150444749213414
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
MechVent
0.013287598918552006
--------------------
Overweight
MechVent
0.012850226540314485
--------------------
Obesity 1
MechVent
0.012875867812014346
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5>Maximum MAE value in each subgroup</h5>

In [24]:
toolkits.max_value_in_subgroup(df_saits_mae_standard, subgroups, variables)

General
AST
400.73789268472706
--------------------
Female
AST
395.8701238334127
--------------------
Male
AST
292.49699600095835
--------------------
Undefined Gender
Urine
57.70387268060635
--------------------
+65
AST
231.16054653347015
--------------------
-65
AST
371.5504835545984
--------------------
ICUType 1
AST
660.6233684539532
--------------------
ICUType 2
AST
213.67784124871028
--------------------
ICUType 3
AST
392.7643692165898
--------------------
ICUType 4
AST
469.25546223032705
--------------------
Undefined classification
AST
346.1022879801262
--------------------
Low Weight
AST
1023.0895042416876
--------------------
Normal Weight
AST
197.998634457582
--------------------
Overweight
AST
522.0102859678475
--------------------
Obesity 1
AST
329.8819475412204
--------------------
Obesity 2
ALT
218.36687033513905
--------------------
Obesity 3
ALT
376.07747713712234
--------------------


<h4>Standard Scaler (S/Normalização)</h4>

In [ ]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard_ori, subgroups, variables)

In [ ]:
df_saits_mae_standard_ori = toolkits.create_table(testing_mae_saits_variables_standard_ori, subgroups, variables)

In [ ]:
df_saits_mae_standard_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_saits_mae_standard_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_saits_mae_standard_ori, subgroups, variables)

<h4>MinMax Scaler (C/Normalização)</h4>

In [25]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_minmax, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 112.21354016661586
ALT : 291.51042278607537
AST : 562.9945693689814
Albumin : 1.9510791413217858
BUN : 24.78843811781201
Bilirubin : 2.7703703989979305
Cholesterol : 129.29999999999356
Creatinine : 1.3927400290337868
DiasABP : 60.684729736472974
FiO2 : 0.3327910102481898
GCS : 8.416553277655188
Glucose : 131.70664855678496
HCO3 : 17.99873046826588
HCT : 25.109207015520656
HR : 86.69561557043218
K : 2.6361485824391617
Lactate : 2.767346140115878
MAP : 80.30970073528962
MechVent : 0.0
Mg : 2.011890264982128
NIDiasABP : 59.44657435889726
NIMAP : 77.73032076927599
NISysABP : 119.4786514127118
Na : 40.88944723618086
PaCO2 : 40.95278192858044
PaO2 : 148.29175956176658
Platelets : 185.6867094356714
RespRate : 19.431791012219442
SaO2 : 96.8508583690985
SysABP : 120.12315150566705
Temp : 54.89088744652194
TroponinI : 8.110000050067496
TroponinT : 0.8610493570602916
Urine : 116.09896736077816
WBC : 12.26488773373189
Weight : 83.7309656814315
P

In [57]:
df_saits_mae_minmax = toolkits.create_table(testing_mae_saits_variables_minmax, subgroups, variables)

In [58]:
df_saits_mae_minmax

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,93.477045,95.923397,94.204229,90.800756,97.736268,112.724208,85.518225,79.479239,127.019654,90.321470,89.091845,157.806162,98.746588,87.399715,82.485978,64.762371,74.295819
1,ALT,311.856460,146.806237,299.865078,876.780331,241.619627,414.153798,340.540936,521.730284,408.045663,300.914797,314.814130,219.491635,189.224936,648.496920,268.583071,776.440910,1629.534974
2,AST,580.753358,632.946314,285.729871,663.955860,279.649209,449.902791,685.770612,355.311073,502.757992,641.779854,307.952064,880.059385,209.915174,682.205256,413.126054,193.328673,148.148190
3,Albumin,1.994266,1.977296,2.006398,2.153890,1.981838,2.007729,2.022257,1.914905,2.005524,2.025073,2.051419,1.967156,1.909736,1.948019,1.956438,1.967108,1.943897
4,BUN,24.293075,23.760087,26.193407,24.147696,27.949329,21.970768,27.157634,20.862990,29.679509,20.261092,25.329214,27.332073,25.181038,24.430181,26.966782,28.234565,26.974401
5,Bilirubin,2.770370,2.151613,2.242935,0.000000,1.634444,3.295798,1.168000,2.325000,2.430097,2.844444,2.758427,8.280000,3.455172,1.586111,2.593333,1.644445,2.216667
6,Cholesterol,117.165723,116.194360,117.916085,123.351923,115.404416,119.041461,115.601955,115.020012,116.757318,120.068165,118.743108,118.607903,113.920232,116.190098,115.977272,118.668994,117.289936
7,Creatinine,1.347340,1.134677,1.479304,1.396137,1.269102,1.310059,1.584878,1.154731,1.681413,1.109242,1.379090,1.009123,1.378736,1.243383,1.278136,1.321840,1.589997
8,DiasABP,62.718084,62.466381,63.073589,65.099445,59.837164,66.474790,61.324229,58.838258,63.913077,65.301539,65.166037,61.570727,60.634648,60.923928,62.216848,62.799902,62.390182
9,FiO2,0.300710,0.294553,0.305745,0.324718,0.300769,0.301508,0.324819,0.312221,0.308347,0.273451,0.299435,0.266262,0.290676,0.303029,0.316807,0.312052,0.311929


<h5>Minimum MAE value in each subgroup</h5>

In [41]:
toolkits.min_value_in_subgroup(df_saits_mae_minmax, subgroups, variables)

General
FiO2
0.5427910102481897
--------------------
Female
FiO2
0.5322841955806464
--------------------
Male
FiO2
0.558764148292676
--------------------
Undefined Gender
TroponinT
0.0
--------------------
+65
FiO2
0.5509840135837525
--------------------
-65
FiO2
0.5440240574521423
--------------------
ICUType 1
FiO2
0.5382165582316663
--------------------
ICUType 2
Cholesterol
0.0
--------------------
ICUType 3
TroponinT
0.4359321938985411
--------------------
ICUType 4
TroponinT
0.4575000104183762
--------------------
Undefined classification
FiO2
0.5253448272903851
--------------------
Low Weight
TroponinI
0.0
--------------------
Normal Weight
FiO2
0.5370249722019451
--------------------
Overweight
FiO2
0.5362397804247246
--------------------
Obesity 1
FiO2
0.5825757534816982
--------------------
Obesity 2
TroponinI
0.0
--------------------
Obesity 3
Cholesterol
0.0
--------------------


<h5> Maximum MAE value in each subgroup </h5>

In [42]:
toolkits.max_value_in_subgroup(df_saits_mae_minmax, subgroups, variables)

General
AST
566.9945693689813
--------------------
Female
AST
625.6842090706995
--------------------
Male
ALT
281.3000011026836
--------------------
Undefined Gender
Urine
139.99999999986
--------------------
+65
AST
239.44805549026773
--------------------
-65
AST
430.2633963482682
--------------------
ICUType 1
AST
683.9200496291841
--------------------
ICUType 2
ALT
497.31578862037054
--------------------
ICUType 3
AST
491.05494833516013
--------------------
ICUType 4
AST
630.3636355139878
--------------------
Undefined classification
ALT
291.7088629807061
--------------------
Low Weight
AST
889.0000677106542
--------------------
Normal Weight
AST
177.37499982118052
--------------------
Overweight
AST
667.7142724082266
--------------------
Obesity 1
AST
388.75000405309646
--------------------
Obesity 2
ALT
772.2857140131256
--------------------
Obesity 3
ALT
1688.3333333327705
--------------------


<h4>MinMax Scaler (S/Normalização)</h4>

In [ ]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_minmax_ori, subgroups, variables)

In [ ]:
df_saits_mae_minmax_ori = toolkits.create_table(testing_mae_saits_variables_minmax_ori, subgroups, variables)

In [ ]:
df_saits_mae_minmax_ori 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_saits_mae_minmax_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_saits_mae_minmax_ori, subgroups, variables)

#### BRITS

<h4> Standard Scaler (C/Normalização)</h4>

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard, subgroups, variables)

In [ ]:
df_brits_mae_standard = toolkits.create_table(testing_mae_brits_variables_standard, subgroups, variables)

In [ ]:
df_brits_mae_standard 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_brits_mae_standard, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_brits_mae_standard, subgroups, variables) 

<h4> Standard Scaler (S/Normalização)</h4>

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard_ori, subgroups, variables)

In [ ]:
df_brits_mae_standard_ori = toolkits.create_table(testing_mae_brits_variables_standard_ori, subgroups, variables)

In [ ]:
df_brits_mae_standard_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_brits_mae_standard_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_brits_mae_standard_ori, subgroups, variables) 

<h4> MinMax Scaler (C/Normalização)</h4>

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_minmax, subgroups, variables)

In [ ]:
df_brits_mae_minmax = toolkits.create_table(testing_mae_brits_variables_minmax, subgroups, variables)

In [ ]:
df_brits_mae_minmax 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_brits_mae_minmax, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_brits_mae_minmax, subgroups, variables) 

<h4> MinMax Scaler (S/Normalização)</h4>

In [ ]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_minmax_ori, subgroups, variables)

In [ ]:
df_brits_mae_minmax_ori = toolkits.create_table(testing_mae_brits_variables_minmax_ori, subgroups, variables)

In [ ]:
df_brits_mae_minmax_ori 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_brits_mae_minmax_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_brits_mae_minmax_ori, subgroups, variables) 

#### USGAN 

<h4> Standard Scaler (C/Normalização)</h4>

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_standard, subgroups, variables)

In [ ]:
df_usgan_mae_standard = toolkits.create_table(testing_mae_saits_variables_standard, subgroups, variables)

In [ ]:
df_usgan_mae_standard 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_usgan_mae_standard, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_usgan_mae_standard, subgroups, variables)

<h4> Standard Scaler (S/Normalização)</h4>

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_standard_ori, subgroups, variables)

In [ ]:
df_usgan_mae_standard_ori = toolkits.create_table(testing_mae_usgan_variables_standard_ori, subgroups, variables)

In [ ]:
df_usgan_mae_standard_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_usgan_mae_standard_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_usgan_mae_standard_ori, subgroups, variables)

<h4>MinMax Scaler (C/Normalização)</h4>

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_minmax, subgroups, variables)

In [ ]:
df_usgan_mae_minmax = toolkits.create_table(testing_mae_usgan_variables_minmax, subgroups, variables)

In [ ]:
df_usgan_mae_minmax

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_usgan_mae_minmax, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_usgan_mae_minmax, subgroups, variables)

<h4>MinMax Scaler (S/Normalização)</h4>

In [ ]:
print("USGAN - MAE")
print("************")
toolkits.show_mae(testing_mae_usgan_variables_minmax_ori, subgroups, variables)

In [ ]:
df_usgan_mae_minmax_ori = toolkits.create_table(testing_mae_usgan_variables_minmax_ori, subgroups, variables)

In [ ]:
df_usgan_mae_minmax_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_usgan_mae_minmax_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_usgan_mae_minmax_ori, subgroups, variables)

#### GPVAE

<h4> Standard Scaler (C/Normalização)</h4>

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_standard, subgroups, variables)

In [ ]:
df_gpvae_mae_standard = toolkits.create_table(testing_mae_gpvae_variables_standard, subgroups, variables)

In [ ]:
df_gpvae_mae_standard 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_gpvae_mae_standard, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_gpvae_mae_standard, subgroups, variables)

<h4> Standard Scaler (S/Normalização)</h4>

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_standard_ori, subgroups, variables)

In [ ]:
df_gpvae_mae_standard_ori = toolkits.create_table(testing_mae_gpvae_variables_standard_ori, subgroups, variables)

In [ ]:
df_gpvae_mae_standard_ori 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_gpvae_mae_standard_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_gpvae_mae_standard_ori, subgroups, variables)

<h4>MinMax Scaler (C/Normalização)</h4>

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_minmax, subgroups, variables)

In [ ]:
df_gpvae_mae_minmax = toolkits.create_table(testing_mae_gpvae_variables_minmax, subgroups, variables)

In [ ]:
df_gpvae_mae_minmax

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_gpvae_mae_minmax, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_gpvae_mae_minmax, subgroups, variables)

<h4>MinMax Scaler (S/Normalização)</h4>

In [ ]:
print("GPVAE - MAE")
print("************")
toolkits.show_mae(testing_mae_gpvae_variables_minmax_ori, subgroups, variables)

In [ ]:
df_gpvae_mae_minmax_ori = toolkits.create_table(testing_mae_gpvae_variables_minmax_ori, subgroups, variables)

In [ ]:
df_gpvae_mae_minmax_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_gpvae_mae_minmax_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_gpvae_mae_minmax_ori, subgroups, variables)

#### MRNN

<h4> Standard Scaler (C/Normalização)</h4>

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_standard, subgroups, variables)

In [ ]:
df_mrnn_mae_standard = toolkits.create_table(testing_mae_mrnn_variables_standard, subgroups, variables)

In [ ]:
df_mrnn_mae_standard 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_mrnn_mae_standard, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_mrnn_mae_standard, subgroups, variables)

<h4> Standard Scaler (S/Normalização)</h4>

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_standard_ori, subgroups, variables)

In [ ]:
df_mrnn_mae_standard_ori = toolkits.create_table(testing_mae_mrnn_variables_standard_ori, subgroups, variables)

In [ ]:
df_mrnn_mae_standard_ori

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_mrnn_mae_standard_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_mrnn_mae_standard_ori, subgroups, variables)

<h4>MinMax Scaler (C/Normalização)</h4>

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_minmax, subgroups, variables)

In [ ]:
df_mrnn_mae_minmax = toolkits.create_table(testing_mae_mrnn_variables_minmax, subgroups, variables)

In [ ]:
df_mrnn_mae_minmax

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_mrnn_mae_minmax, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_mrnn_mae_minmax, subgroups, variables)

<h4>MinMax Scaler (S/Normalização)</h4>

In [ ]:
print("MRNN - MAE")
print("************")
toolkits.show_mae(testing_mae_mrnn_variables_minmax_ori, subgroups, variables)

In [ ]:
df_mrnn_mae_minmax_ori = toolkits.create_table(testing_mae_mrnn_variables_minmax_ori, subgroups, variables)

In [ ]:
df_mrnn_mae_minmax_ori 

<h5>Minimum MAE value in each subgroup</h5>

In [ ]:
toolkits.min_value_in_subgroup(df_mrnn_mae_minmax_ori, subgroups, variables)

<h5>Maximum MAE value in each subgroup</h5>

In [ ]:
toolkits.max_value_in_subgroup(df_mrnn_mae_minmax_ori, subgroups, variables)